<a href="https://colab.research.google.com/github/athira-tech1/athira-tech1/blob/main/videointegrityframework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python cryptography


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving SampleVideo_1280x720_2mb.mp4 to SampleVideo_1280x720_2mb.mp4


In [ ]:
import cv2
import hashlib

def extract_and_hash_frames(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    frame_hashes = []
    frame_count = 0

    # Read and process each frame
    while video.isOpened():
        ret, frame = video.read()

        if not ret:
            break  # Exit the loop if no more frames are found

        # Convert the frame to JPEG format in memory
        frame_bytes = cv2.imencode('.jpg', frame)[1].tobytes()

        # Generate SHA-256 hash of the frame
        frame_hash = hashlib.sha256(frame_bytes).hexdigest()

        # Store the frame hash
        frame_hashes.append((frame_count, frame_hash))
        frame_count += 1

    # Release the video capture object
    video.release()

    # Return the list of frame hashes
    return frame_hashes

# Specify the video path
video_path = 'SampleVideo_1280x720_2mb.mp4'

# Run the function and get the hashes
frame_hashes = extract_and_hash_frames(video_path)

# Display the frame hashes
for frame_number, frame_hash in frame_hashes[:5]:  # Displaying the first 5 for brevity
    print(f"Frame {frame_number}: Hash = {frame_hash}")


Frame 0: Hash = aaa510d3f8ecf25fd42c8de2d77adb67db5bd91f10cb8e4bef8a93b4cc2902bb
Frame 1: Hash = c25636809722889068310ad26fd5aa9016f6e8f1889d5aced5b958f114a5e293
Frame 2: Hash = 027eb9fba393e612ce11c1bc28ea40a41cff01c6fa0f6e72b1a70a2f33de6896
Frame 3: Hash = 72c5e06ed26dafd2593acb1bcac2a323244a5bd7a6eb86a5aa5872ae21fa7225
Frame 4: Hash = 1b6576306b75bd80c05b2af74149250c3ceab6fca29f67d25b4f4c017eaa8af9


In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization

# Generate RSA keys (for demonstration; in practice, securely load existing keys)
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

def sign_hashes(frame_hashes, private_key):
    signed_hashes = []
    for frame_hash in frame_hashes:
        signature = private_key.sign(
            frame_hash.encode(),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        signed_hashes.append((frame_hash, signature))
    return signed_hashes


In [ ]:
def verify_frame_hashes(signed_hashes, public_key):
    for frame_hash, signature in signed_hashes:
        try:
            public_key.verify(
                signature,
                frame_hash.encode(),
                padding.PSS(
                    mgf=padding.MGF1(hashes.SHA256()),
                    salt_length=padding.PSS.MAX_LENGTH
                ),
                hashes.SHA256()
            )
        except:
            print("Frame hash verification failed!")
            return False
    print("All frames verified successfully.")
    return True
